In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Input, concatenate
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
df = pd.read_csv(r"/Users/priyankapalaniselvam/Downloads/classifier_one_data/full_df.csv")
if 'ID' in df.columns:
    df = df.drop('ID', axis=1)

In [3]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['labels'] = label_encoder.fit_transform(df['labels'])
df['labels'] = df['labels'].astype(str)
left_images = df[['Left-Fundus', 'labels']]
right_images = df[['Right-Fundus', 'labels']]

left_images.columns = ['image', 'target']
right_images.columns = ['image', 'target']

combined_df = pd.concat([left_images, right_images])
combined_df = combined_df.dropna(subset=['image'])

train_df, test_df = train_test_split(combined_df, test_size=0.2, random_state=42)


In [4]:
datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='/Users/priyankapalaniselvam/Downloads/classifier_one_data/ODIR-5K/ODIR-5K/Training Images',
    x_col='image',
    y_col='target',
    class_mode="categorical",
    target_size=(224, 224),
    batch_size=16
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='/Users/priyankapalaniselvam/Downloads/classifier_one_data/ODIR-5K/ODIR-5K/Training Images',
    x_col="image",
    y_col="target",
    class_mode="categorical",
    target_size=(224, 224),
    batch_size=32
)

Found 10227 validated image filenames belonging to 8 classes.
Found 2557 validated image filenames belonging to 8 classes.


In [7]:
input_shape = (224, 224, 3)
input_layer = Input(shape=input_shape)
num_classes = 8
base_model = Xception(include_top=False, weights='imagenet', input_tensor=input_layer)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
output_layer = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=input_layer, outputs=output_layer)

In [8]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    # callbacks=[early_stopping, checkpoint]
)

Epoch 1/20


/opt/anaconda3/envs/test310/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


640/640 ━━━━━━━━━━━━━━━━━━━━ 9609s 15s/step - accuracy: 0.4714 - loss: 1.4946 - val_accuracy: 0.5823 - val_loss: 1.1486
Epoch 2/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 17163s 27s/step - accuracy: 0.6266 - loss: 1.0264 - val_accuracy: 0.6019 - val_loss: 1.0887
Epoch 3/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 7799s 12s/step - accuracy: 0.7254 - loss: 0.7552 - val_accuracy: 0.6672 - val_loss: 1.0064
Epoch 4/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 7287s 11s/step - accuracy: 0.7890 - loss: 0.5821 - val_accuracy: 0.6750 - val_loss: 1.0438
Epoch 5/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 6198s 10s/step - accuracy: 0.8248 - loss: 0.4666 - val_accuracy: 0.6883 - val_loss: 1.0926
Epoch 6/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 10042s 16s/step - accuracy: 0.8458 - loss: 0.3903 - val_accuracy: 0.6711 - val_loss: 1.0880
Epoch 7/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 5296s 8s/step - accuracy: 0.8500 - loss: 0.3415 - val_accuracy: 0.6723 - val_loss: 1.2231
Epoch 8/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 5831s 9s/step - accuracy: 0.8487 - loss: 0.3181 - val

In [13]:
model.save_weights('base_model_exc.weights.h5')
